## TIRCP Calsta
* California State Transportation Agency emailed a request asking for TIRCP outcomes for cycles 3-5.  
* [Cycles 1-6](https://calsta.ca.gov/subject-areas/transit-intercity-rail-capital-prog)
* Cycle 1: 2015
* Cycle 2: 2016
* Cycle 3: 2018
* Cycle 4: 2020
* Cycle 5: 2022
* Cycle 6: 2023

In [30]:
import A1_data_prep
import A2_tableau
import A6_zev
import numpy as np
import pandas as pd
from babel.numbers import format_currency
from calitp import *

/opt/conda/lib/python3.9/site-packages/geopandas/_compat.py:123: UserWarning: The Shapely GEOS version (3.10.3-CAPI-1.16.1) is incompatible with the GEOS version PyGEOS was compiled with (3.10.1-CAPI-1.16.0). Conversions between both will be slow.
  warnings.warn(


In [31]:
pd.options.display.max_columns = 100
pd.options.display.float_format = "{:.2f}".format
pd.set_option("display.max_rows", None)
pd.set_option("display.max_colwidth", None)

In [32]:
# GCS File Path:
GCS_FILE_PATH = "gs://calitp-analytics-data/data-analyses/tircp/"

### Manipulate TIRCP
#### Filter out for cycles of interest

In [33]:
df_tircp = to_snakecase(A2_tableau.tableau_dashboard())

/opt/conda/lib/python3.9/site-packages/openpyxl/worksheet/_reader.py:312: UserWarning: Data Validation extension is not supported and will be removed


In [34]:
# Drop irrelevant years. 
df_tircp2 = df_tircp.loc[df_tircp["award_year"] >= 2018].reset_index(drop=True)

In [35]:
# Sort df by award year and number
df_tircp2 = df_tircp2.sort_values(["award_year", "#"])

In [36]:
# Double check that the right cycles are selected. 
df_tircp2.award_year.value_counts(),

(2018    28
 2022    23
 2020    17
 Name: award_year, dtype: int64,)

In [37]:
df_tircp2.ppno.nunique(), df_tircp2.title.nunique(), len(df_tircp2)

(66, 67, 68)

#### Duplicate Project Title Fix

In [38]:
# Find duplicate project title 
df_tircp2.title.value_counts().head()

North State Intercity Bus System                                                                    2
Purchase Zero Emission High Capacity Buses to Support Transbay Tomorrow and Clean Corridors Plan    1
Expansion of WETA Ferry Services                                                                    1
South Bay Microtransit Expansion                                                                    1
Sacramento Valley Station (SVS) Transit Center: Priority Project                                    1
Name: title, dtype: int64

In [39]:
# Create a detailed title column to avoid duplicates by combining  year,title, and recipient into one line.
df_tircp2["award_year"] = df_tircp2["award_year"].astype("object")

In [40]:
# https://stackoverflow.com/questions/39291499/how-to-concatenate-multiple-column-values-into-a-single-column-in-pandas-datafra'
detailed_title_cols = [
    "award_year",
    "title",
    "grant_recipient",
]

In [41]:

df_tircp2["detailed_title_col"] = df_tircp2[detailed_title_cols].apply(
    lambda row: "-".join(row.values.astype(str)), axis=1
)

In [42]:
df_tircp2.columns

Index(['award_year', '#', 'grant_recipient', 'title', 'ppno', 'district',
       'county', 'description', 'master_agreement_number',
       'master_agreement_expiration_date', 'manager', 'regional_coordinator',
       'technical_assistance_calitp__y_n', 'technical_assistance_fleet__y_n',
       'technical_assistance_network_integration__y_n',
       'technical_assistance_priority_population__y_n', 'total__cost', 'tircp',
       'allocated_amount', 'unallocated_amount', 'percentage_allocated',
       'expended_amount', 'other_funds_involved', 'award_cycle', 'is_sb1?',
       'is_ggrf?', 'is_iija?', 'on_shs?', 'calitp',
       'estimated_tircp_ghg_reductions', 'estimated_tircp_ghg_reductions2',
       'increased_ridership', 'service_integration', 'improve_safety',
       'readiness', 'funding_leverage',
       'multi_agency_coordination_integration', 'ab_1550_community_benefits',
       'housing_co_benefits', 'local_agency_address', 'local_agency_city',
       'local_agency_zip', 'local_

In [43]:
# Subset df to columns of interest
df_tircp2 = df_tircp2[
    [
        "award_year",
        "#",
        "ppno",
        "tircp",
        "title",
        "detailed_title_col",
        "grant_recipient",
        "district",
        "county",
        "description",
        "total__cost",
        "estimated_tircp_ghg_reductions",
        "increased_ridership",
        "service_integration",
        "improve_safety",
    ]
]

#### Add Project Number
* Year + Number

In [44]:
df_tircp2["project_number_use"] = (
    df_tircp2["award_year"].astype(str) + "-" + df_tircp2["#"].astype(str)
)

In [45]:
# df_tircp2[["project_number_use","award_year","#","title"]]

### Recreate SCCP's output example with TIRCP data.
Columns to include: 

> Project ID	Project Name	Implementing Agency	Program	Project Description	 Total Cost 	 SB 1 Funds 	Fiscal Year	Is SB 1?	Project Status	Assembly Districts	Senate Districts	Counties	Cities	Caltrans Districts	Is on SHS?	Date Updated	Cycle


#### GIS Template 
* Although Linda provided me with more updated/complete GIS information directly from the Tracking Sheet 2.0, using this to glean project statuses. 

In [46]:
# Read in sheet with Assembly info.
gis = to_snakecase(
    pd.read_excel(
        f"{GCS_FILE_PATH}TIRCP_GIS_Template_Requirements 6-1-2022.xlsx",
        sheet_name="Projects Table",
    )
)

In [47]:
# Clean some column names
gis = gis.rename(
    columns={
        "ppno_": "ppno",
    }
)

In [48]:
# Clean PPNO
gis = A1_data_prep.ppno_slice(gis)

In [49]:
# Subset for only cols of interest
gis2 = gis[
    [
        "project_number",
        "ppno",
        "projecttitle",
        "projectstatus",
    ]
]

In [50]:
gis2.ppno.nunique()

45

In [51]:
# There are mulitple entries for each ppno.
gis2.ppno.value_counts().head()

CP033    60
CP035    21
CP042    18
CP032    14
CP031    11
Name: ppno, dtype: int64

In [52]:
# Inglewood Transit Center coded as CP063, should be CP062 per TIRCP Tracking sheet.
gis2.loc[
    (gis2["projecttitle"] == "Inglewood Transit Center (2020:04)"), "ppno"
] = "CP062"

/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:1732: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/opt/conda/lib/python3.9/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [53]:
# North State Intercity Bus System coded as CP063 per TIRCP Tracking sheet.
gis2.loc[
    (
        gis2["projecttitle"]
        == "North State Intercity Bus System-Lake County Interregional Transit Center (2020:05)"
    ),
    "ppno",
] = "CP063"

In [54]:
# Clean project_number, only keep year
gis2["project_number"] = gis2["project_number"].str.split(":").str[0]

/tmp/ipykernel_2472/1194175043.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [55]:
gis2["project_number"] = gis2["project_number"].fillna(0).astype("int64")

/tmp/ipykernel_2472/511736485.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [56]:
project_status_gis = A6_zev.summarize_rows(gis2, "ppno", "projectstatus")

In [57]:
# Check that each row matches the number of unique ppno
len(project_status_gis) == gis2.ppno.nunique()

True

In [58]:
# Drop old project status from original dataframe
gis2 = gis2.drop(columns=["projectstatus"])

In [59]:
# Merge with original gis, so there is only one row for each PPNO
final_gis_template = (
    pd.merge(project_status_gis, gis2, how="left", on=["ppno"])
    .drop_duplicates("ppno")
    .reset_index(drop=True)
)

In [60]:
# Checks
len(final_gis_template), final_gis_template.ppno.nunique(), gis.ppno.nunique()

(45, 45, 45)

##### Merge GIS information 

In [61]:
# Merge with df_tircp2
merge1 = pd.merge(
    df_tircp2,
    final_gis_template,
    how="left",
    left_on=["ppno", "award_year"],
    right_on=["ppno", "project_number"],
    indicator=True,
)

In [62]:
merge1._merge.value_counts()

both          43
left_only     25
right_only     0
Name: _merge, dtype: int64

In [63]:
# Double Check that titles & years correspond with one another
merge1[["title", "projecttitle","projectstatus"]].sample(2)

,title,projecttitle,projectstatus
50,Making a Beeline for Electrification - City of Glendale and Arroyo Verdugo Communities Zoom towards Cleaner Transportation,NaN,NaN
12,Peninsula Corridor Electrification Expansion Project,Peninsula Corridor Electrification Expansion (2018:13),"PS&E, Ops./Procure, Construction"


In [64]:
# Drop merge & GIS Template projects
merge1 = merge1.drop(columns=["_merge", "projecttitle"])

#### GIS Info from Tracking Sheet 2.0 

In [65]:
gis_tracking_sheet = to_snakecase(
    pd.read_excel(f"{GCS_FILE_PATH}{A1_data_prep.FILE_NAME }", sheet_name="GIS Info")
)

In [66]:
# Drop certain cols
gis_tracking_sheet = gis_tracking_sheet[
    [
        "award_year",
        "project_title",
        "caltransdistrict",
        "assembly\ndistricts",
        "senate\ndistricts",
        "city_code",
        "county_code",
        "_implementing_agency__id_",
    ]
]

In [67]:
# Only keep the years wanted
gis_tracking_sheet2 = gis_tracking_sheet.loc[
    gis_tracking_sheet["award_year"] >= 2018
].reset_index(drop=True)

In [68]:
# Merge with df_tircp2
merge2 = pd.merge(
    merge1,
    gis_tracking_sheet2,
    how="left",
    left_on=["award_year", "title"],
    right_on=["award_year", "project_title"],
    indicator=True,
)

In [69]:
# Merge
merge2._merge.value_counts()

both          68
left_only      0
right_only     0
Name: _merge, dtype: int64

In [70]:
merge2.shape, df_tircp2.shape

((68, 26), (68, 16))

In [71]:
merge2 = merge2.drop(
    columns=["project_number", "project_title", "_merge"]
)

### Project Sheet 
* Recreating "projects" tab of SCCP Excel workbook. 

In [72]:
# Copy merge 2
projects = merge2.copy()

In [73]:
# Fill in empty values with NA
projects = projects.fillna(
    projects.dtypes.replace({"float64": 0.0, "object": "None", "int64": 0})
)

In [74]:
# Format moentary cols
monetary_cols = ["total__cost", "tircp"]
for i in monetary_cols:
    projects[i] = projects[i].apply(
        lambda x: format_currency(x, currency="USD", locale="en_US")
    )

In [75]:
# Clean up column names 
projects = A1_data_prep.clean_up_columns(projects)

In [76]:
projects = projects.rename(
    columns={
        "Number Use": "Project Number",
        "Assembly\nDistricts": "Assembly Districts",
        "Senate\nDistricts": "Senate Districts",
        "Caltransdistrict": "CT Districts",
        "Assembly\nDistricts": "Assembly Districts",
    }
)

In [77]:
# Rearrange columns
right_order = [
    "Award Year",
    "#",
    "Project Number",
    "Ppno",
    "Title",
    "Grant Recipient",
    "Tircp",
    "Total  Cost",
    "Description",
    "District",
    "County",
    "Status",
    "CT Districts",
    "Assembly Districts",
    "Senate Districts",
    "City Code",
    "County Code",
    "Implementing Agency  Id",
]

In [78]:
projects = projects[right_order]

### Outcomes Sheet

In [79]:
# Measure columns
measure_cols = [
    "estimated_tircp_ghg_reductions",
    "cost_per_ghg_ton_reduced",
    "increased_ridership",
    "service_integration",
    "improve_safety",
]

In [80]:
# Turn estimated GHG reductions into a number
merge2["estimated_tircp_ghg_reductions"] = (
    merge2["estimated_tircp_ghg_reductions"]
    .str.replace("MTCO2e", "")
    .str.replace("None", "")
    .str.replace(",", "")
)

In [81]:
merge2["estimated_tircp_ghg_reductions"] = (
    merge2["estimated_tircp_ghg_reductions"]
    .apply(pd.to_numeric, errors="coerce")
    .fillna(0)
)

In [83]:
# Subset to cols similar to SCCP
outcomes = merge2[
    [
        "award_year",
        'detailed_title_col',
        "estimated_tircp_ghg_reductions",
        "increased_ridership",
        "service_integration",
        "improve_safety",
    ]
].sort_values(["award_year", 'detailed_title_col',])

In [84]:
outcomes = A1_data_prep.clean_up_columns(outcomes)

In [85]:
outcomes.head(1)

,Award Year,Detailed Title Col,Estimated Tircp Ghg Reductions,Increased Ridership,Service Integration,Improve Safety
1,2018,2018-#Electrify Anaheim: Changing the Transit Paradigm in Southern California-Anaheim Transportation Network,61000.00,Medium-High,Medium-High,Medium


##### Version 1

In [86]:
# Drop award year
outcomes_transformed = outcomes.drop(columns=["Award Year"]).T

In [87]:
# Make first row to column names
outcomes_transformed.columns = outcomes_transformed.iloc[0]

In [88]:
# Del first row
outcomes_transformed = outcomes_transformed.iloc[1:]

##### Outputs: Measures except GHG Reductions.

In [90]:
outcomes_melt = pd.melt(
    outcomes,
    id_vars=[
        "Award Year",
        "Detailed Title Col",
    ],
    value_vars=[
        "Increased Ridership",
        "Service Integration",
        "Improve Safety",
    ],
)

In [91]:
outcomes_melt = A1_data_prep.clean_up_columns(outcomes_melt)

In [92]:
year_summary = (
    outcomes_melt.groupby(["Award Year", "Variable", "Value"])
    .agg({"Detailed Title Col": "nunique"})
    .rename(
        columns={"Detailed Title Col": "Number of Projects in this Value Category"}
    )
)

In [93]:
year_summary

Number of Projects in this Value Category
Award Year Variable            Value                                                 
2018       Improve Safety      High                                                 9
                               Medium                                              12
                               Medium-High                                          7
           Increased Ridership High                                                13
                               Medium                                              10
                               Medium-High                                          5
           Service Integration High                                                 5
                               Medium                                               8
                               Medium-High                                         15
2020       Improve Safety      High                                                 4
                               Medium                                               7
                               Medium-High                                          6
           Increased Ridership High                                                 1
                               Medium                                               4
                               Medium-High                                          9
                               Medium-Low                                           3
           Service Integration High                                                 2
                               Medium                                               5
                               Medium-High                                          8
                               Medium-Low                                           2
2022       Improve Safety      High                                                 8
                               Medium                                               8
                               Medium-High                                          6
                               Medium-Low                                           1
           Increased Ridership High                                                 6
                               Medium                                               7
                               Medium-High                                          8
                               Medium-Low                                           2
           Service Integration High                                                 5
                               Medium                                               8
                               Medium-High                                          7
                               Medium-Low                                           3

##### GHG Reductions.

In [94]:
GHG_by_year = outcomes.groupby(["Award Year"]).agg(
    {"Estimated Tircp Ghg Reductions": "sum"}
)

In [95]:
GHG_by_year

,Estimated Tircp Ghg Reductions
Award Year,
2018,31944000.00
2020,5016000.00
2022,4332000.00


#### Save

In [96]:
"""
with pd.ExcelWriter(f"{GCS_FILE_PATH}calsta_draft.xlsx") as writer:
    outcomes.to_excel(writer, sheet_name="outcomes_unpivoted", index=True)
    outcomes_transformed.to_excel(writer, sheet_name="outcomes_transformed", index=True)
    projects.to_excel(writer, sheet_name="projects", index=True)
    year_summary.to_excel(writer, sheet_name="year_summary", index=True)
    GHG_by_year.to_excel(writer, sheet_name="GHG_reduction_year", index=True)
    """

'\nwith pd.ExcelWriter(f"{GCS_FILE_PATH}calsta_draft.xlsx") as writer:\n    outcomes.to_excel(writer, sheet_name="outcomes_unpivoted", index=True)\n    outcomes_transformed.to_excel(writer, sheet_name="outcomes_transformed", index=True)\n    projects.to_excel(writer, sheet_name="projects", index=True)\n    year_summary.to_excel(writer, sheet_name="year_summary", index=True)\n    GHG_by_year.to_excel(writer, sheet_name="GHG_reduction_year", index=True)\n    '